<a href="https://colab.research.google.com/github/srirampattabiraman/Extensive-NLP-and-Deep-Learning/blob/main/session_5/Standford_Data_Augumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#We are mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
##Installing Necessary Packages
!pip install googletrans==3.1.0a0

# import googletrans.Translator

     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 1.0MB 10.9MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=0de9468bd18552fdcba476d5e065ec940d119c05a7bab9784c5ef703f5cd869f
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [5]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.legacy import data
from torchtext.legacy.datasets import IMDB
from torchtext.legacy.data import Field, LabelField, BucketIterator
import random
import googletrans
from googletrans import Translator
from tqdm import tqdm
import sys
import os

In [6]:
def group_labels(label):
  if label in ["very negative", "negative"]:
    return "negative"
  elif label in ["positive", "very positive"]:
    return "positive" 
  else:
    return "neutral"

In [8]:
dictionary = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/dictionary.txt", sep="|")
dictionary.columns = ["phrase", "id"]
dictionary = dictionary.set_index("id")

sentiment_labels = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/sentiment_labels.txt", sep="|")
sentiment_labels.columns = ["id", "sentiment"]
sentiment_labels = sentiment_labels.set_index("id")

phrase_sentiments = dictionary.join(sentiment_labels)

phrase_sentiments["fine"] = pd.cut(phrase_sentiments.sentiment, [0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                           include_lowest=True,
                                           labels=["very negative", "negative", "neutral", "positive", "very positive"])
phrase_sentiments["coarse"] = phrase_sentiments.fine.apply(group_labels)

In [13]:
phrase_sentiments = phrase_sentiments.reset_index(level=0)

In [14]:
sentences = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/datasetSentences.txt", index_col="sentence_index",
                                sep="\t")
splits = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/datasetSplit.txt", index_col="sentence_index")
sentence_partitions = sentences.join(splits)

In [20]:
data = sentence_partitions.join(phrase_sentiments.set_index("phrase"), on="sentence")
data["splitset_label"] = data["splitset_label"].fillna(1).astype(int)
split_data = data.groupby("splitset_label")

In [16]:

def back_translate(sequence,lab, PROB = 1):
    languages = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi',
                 'sw', 'vi', 'es', 'el']
    
    #instantiate translator
    translator = Translator()
    
    #store original language so we can convert back
    org_lang = translator.detect(sequence).lang
    
    #randomly choose language to translate sequence to  
    random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])
    #print(random_lang)
    if org_lang in languages:
        #translate to new language and back to original
        translated = translator.translate(sequence, dest = random_lang).text
        #translate back to original language
        translated_back = translator.translate(translated, dest = org_lang).text
        #print(translated,translated_back)
        #apply with certain probability
        if np.random.uniform(0, 1) <= PROB:
            output_sequence = translated_back
        else:
            output_sequence = sequence
            
    #if detected language not in our list of languages, do nothing
    else:
        output_sequence = sequence
    
    return output_sequence,lab

In [18]:
def random_deletion(words,lab, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] ,lab
    else:
        return remaining,lab

In [19]:
def random_swap(sentence,lab, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence,lab

In [23]:
for splitset, partition in split_data:
    split_name = {1: "train", 2: "test", 3: "dev"}[splitset]
    filename = split_name+'.csv'
    del partition["splitset_label"]
    partition.to_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/"+filename, index=False)

In [22]:
def discretize_label(label):
    if label <= 0.05*100: return 'Class1'
    if label <= 0.1*100: return 'Class2'
    if label <= 0.15*100: return 'Class3'
    if label <= 0.2*100: return 'Class4'
    return 'Class5'

In [24]:
train_data = pd.read_csv('drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/train.csv')
test_data = pd.read_csv('drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/test.csv')
dev_data = pd.read_csv('drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/dev.csv')

In [26]:
Train_df = pd.concat([train_data,test_data])
Train_df.reset_index(inplace = True,drop = True)

Train_df.shape

(10754, 5)

In [27]:
def random_pick(df):
    for num in range (df.shape[0]):
        return df.sentence[num],df.sentiment[num]

In [30]:
df = Train_df.copy()
pbar = tqdm(range(5492+1057,df.shape[0]))
count = len(df)
print (f'Before the shape was :{len(df)}' )

aug_data = []
aug_label = []
for i in pbar:
    
    
    word,val = random_pick(df)
    word1,val1 = back_translate(word,val)
    
    word,val = random_pick(df)
    word = word.split()
    word2,val2 = random_deletion(word,val)
    word2 = ' '.join(i for i in word2)
    
    word,val = random_pick(df)
    word = word.split()
    word3,val3 = random_swap(word,val)
    word3 = ' '.join(i for i in word3)
     
    ins = {'sentence':[word1,word2,word3],'label':[val1,val2,val3]}
    df2 = pd.DataFrame(ins)
    df2.to_csv('transforms_1.csv')
pbar.set_description(desc = f'Loop:{i}')


  7%|▋         | 278/4205 [02:30<1:34:10,  1.44s/it]

KeyboardInterrupt: ignored